In [ ]:
# !pip install transformers datasets comet-ml --yes

In [155]:

import datetime
hug_model_link = "simecek/DNADeberta"
exp_start = "{:%Y-%b-%d_%H:%M:%S}".format(datetime.datetime.now())
slash_index = hug_model_link.index("/")
file_name = hug_model_link[:slash_index] + '_' + hug_model_link[slash_index+1:] + '.csv'

# fields = ['Dataset', 'F1', 'Acc'] 
# with open(model_name + exp_start, 'w') as f:
with open(file_name, 'a+') as f:
    print("open")
    write = csv.writer(f)  
    write.writerow([hug_model_link, exp_start])
file_name

open


'simecek_DNADeberta.csv'

In [159]:
from statistics import stdev, fmean

genomic_datasets = list_datasets()
genomic_datasets.sort()
print(genomic_datasets)
dataset_iterations = 1
csv_rows=[]

for dataset_name in genomic_datasets:
    for k in range(dataset_iterations):
        f1, acc = 5, 10
        csv_rows.append([dataset_name, f1, acc])

stand_devs_header = ["dataset", "f1 mean", "f1 sd", "acc mean", "acc sd"]
stand_devs = []
for dataset_index in range(len(genomic_datasets)):
    f_scores = [csv_rows[i][1] for i in range(dataset_iterations)]
    acc_scores = [csv_rows[i][2] for i in range(dataset_iterations)]
    stand_devs.append([
        genomic_datasets[dataset_index], 
        fmean(f_scores), stdev(f_scores), 
        fmean(acc_scores), stdev(acc_scores)
    ])

print(stand_devs)


['demo_coding_vs_intergenomic_seqs', 'demo_human_or_worm', 'dummy_mouse_enhancers_ensembl', 'human_enhancers_cohn', 'human_enhancers_ensembl', 'human_ensembl_regulatory', 'human_nontata_promoters', 'human_ocr_ensembl']


StatisticsError: variance requires at least two data points

In [158]:
[print(info(dataset)) for dataset in genomic_datasets]

/home/jovyan/my-conda-envs/myEnv/lib/python3.8/site-packages/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


Dataset `demo_coding_vs_intergenomic_seqs` has 2 classes: coding_seqs, intergenomic_seqs.

All lengths of genomic intervals equals 200.

Totally 100000 sequences have been found, 75000 for training and 25000 for testing.
                   train   test
coding_seqs        37500  12500
intergenomic_seqs  37500  12500


/home/jovyan/my-conda-envs/myEnv/lib/python3.8/site-packages/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


Dataset `demo_human_or_worm` has 2 classes: human, worm.

All lengths of genomic intervals equals 200.

Totally 100000 sequences have been found, 75000 for training and 25000 for testing.
       train   test
human  37500  12500
worm   37500  12500


/home/jovyan/my-conda-envs/myEnv/lib/python3.8/site-packages/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


...

/home/jovyan/my-conda-envs/myEnv/lib/python3.8/site-packages/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


Dataset `human_nontata_promoters` has 2 classes: negative, positive.

All lengths of genomic intervals equals 251.

Totally 36131 sequences have been found, 27097 for training and 9034 for testing.
          train  test
negative  12355  4119
positive  14742  4915


/home/jovyan/my-conda-envs/myEnv/lib/python3.8/site-packages/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


Dataset `human_ocr_ensembl` has 2 classes: negative, positive.

The length of genomic intervals ranges from 71 to 593, with average 326.3452470873675 and median 315.0.

Totally 174756 sequences have been found, 139804 for training and 34952 for testing.
          train   test
negative  69902  17476
positive  69902  17476


[None, None, None, None, None, None, None, None]

In [91]:
import os

commet_key = "EpKIINrla6U4B4LJhd9Sv4i0b"

os.environ['COMET_API_KEY'] = commet_key

import comet_ml

# Commet Init
comet_ml.init(project_name="DNA_finetuning", api_key= commet_key)

import os
import comet_ml
import torch 
import datasets 
import numpy as np
import pandas as pd
from pathlib import Path
from datasets import Dataset, DatasetDict, load_metric
from huggingface_hub import notebook_login
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer, EarlyStoppingCallback
from genomic_benchmarks.dataset_getters.pytorch_datasets import HumanNontataPromoters
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import list_datasets


COMET INFO: Comet API key is valid
COMET INFO: Comet API key saved in /home/jovyan/.comet.config


In [48]:
genomic_datasets = list_datasets()
genomic_datasets.sort()
genomic_datasets.remove("human_ocr_ensembl")
print(genomic_datasets)

['demo_coding_vs_intergenomic_seqs', 'demo_human_or_worm', 'dummy_mouse_enhancers_ensembl', 'human_enhancers_cohn', 'human_enhancers_ensembl', 'human_ensembl_regulatory', 'human_nontata_promoters']


In [49]:
# config
epochs = 10
model_name = "DNADeberta_fine"
dataset_name = "demo_coding_vs_intergenomic_seqs"
POSITIVE_CLASS_INDEX = 1

info(dataset_name)

/home/jovyan/my-conda-envs/myEnv/lib/python3.8/site-packages/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


Dataset `demo_coding_vs_intergenomic_seqs` has 2 classes: coding_seqs, intergenomic_seqs.

All lengths of genomic intervals equals 200.

Totally 100000 sequences have been found, 75000 for training and 25000 for testing.


,train,test
coding_seqs,37500,12500
intergenomic_seqs,37500,12500


In [50]:
print(torch.cuda.get_device_name(0))

NVIDIA A40


In [51]:
# notebook_login()

In [52]:
# download_dataset(dataset_name, version=0)

In [53]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("armheb/DNA_bert_6")

loading configuration file https://huggingface.co/armheb/DNA_bert_6/resolve/main/config.json from cache at /home/jovyan/.cache/huggingface/transformers/2697389de18c4fe8c3497cea35aaf65130fdd59c3ab64cb6b1c2e0632fefdaf0.3a7e1ca237211e6405270f85616f49989aeee994db35f6593a40c7b5081a50d0
Model config BertConfig {
  "_name_or_path": "armheb/DNA_bert_6",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_ids": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_rnn_layer": 1,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "rnn": "lstm",
  "rnn_dropout": 0.0,
  "rnn_hidden": 768,
  "split": 10,
  "transformers_version": "4.20.0",

In [54]:
def kmers(s, k=6):
    return [s[i:i + k] for i in range(0, len(s), k) if i + k <= len(s)]

In [55]:
# for dset in ['train', 'test']:
#   for c in ['negative', 'positive']:
#     for f in Path(f'/home/jovyan/.genomic_benchmarks/human_nontata_promoters/{dset}/{c}/').glob('*.txt'):
#       txt = f.read_text()
#       tmp_dict[f.stem] = (dset, int(c == "positive"), txt)

tmp_dict = {}
debug_tmp=[]

print(dataset_name)

for dset in ['train', 'test']:
    list_dir = os.listdir(Path(f'/home/jovyan/.genomic_benchmarks/{dataset_name}/{dset}/'))
    print(list_dir)
    print(dset)
    print(type(list_dir))
    for c in list_dir:
        print(c)
        for f in Path(f'/home/jovyan/.genomic_benchmarks/{dataset_name}/{dset}/{c}/').glob('*.txt'):
            txt = f.read_text()
            debug_tmp.append([int(c == list_dir[1]), c, list_dir[POSITIVE_CLASS_INDEX], f.stem + c])

            # f.stem + c            
            # temporal solution to fix the problem with repeated indexes in positive and negative class (index X can be found in both classes, for each index X)
            tmp_dict[f.stem + c] = (dset, int(c == list_dir[POSITIVE_CLASS_INDEX]), txt)

demo_coding_vs_intergenomic_seqs
['coding_seqs', 'intergenomic_seqs']
train
<class 'list'>
coding_seqs
intergenomic_seqs
['coding_seqs', 'intergenomic_seqs']
test
<class 'list'>
coding_seqs
intergenomic_seqs


In [60]:
my_list = [debug_tmp[x][3] for x in range(len(debug_tmp))] 
print(my_list[0])
indices = [i for i, x in enumerate(my_list) if x == my_list[0]]
print(indices)

21770coding_seqs
[0]


In [57]:
print(list(tmp_dict.values())[-1][1])
print(list(tmp_dict.values())[0][1])
print(debug_tmp[-1])
print(debug_tmp[0])
print(len(tmp_dict))
print(len(debug_tmp))

if(list(tmp_dict.values())[-1][1] != debug_tmp[-1]):
    print("Something is wring with the dataset processing")
    
if(list(tmp_dict.values())[0][1] != debug_tmp[0]):
    print("_____ Something is wring with the dataset processing ________")
    
if(len(tmp_dict) != len(debug_tmp)):
    print("Something is wring with the dataset processing ________")

1
0
[1, 'intergenomic_seqs', 'intergenomic_seqs', '12212intergenomic_seqs']
[0, 'coding_seqs', 'intergenomic_seqs', '21770coding_seqs']
100000
100000
Something is wring with the dataset processing
_____ Something is wring with the dataset processing ________


In [58]:
tmplist = list(tmp_dict.values())
tmplist = [tmplist[x][1] for x in range(len(tmplist))]
print(tmplist[1])
print(len(tmplist))
print(np.all([np.array(tmplist) == 1]))

0
100000
False


In [59]:
debuglist = [debug_tmp[x][0] for x in range(len(debug_tmp))]
print(debuglist[1])
print(len(debuglist))
print(np.all([np.array(debuglist) == 1]))

0
100000
False


In [61]:

df = pd.DataFrame.from_dict(tmp_dict).T.rename(columns = {0: "dset", 1: "cat", 2: "seq"})

In [62]:
print(np.all([df['cat'].to_numpy() == 1]))

False


In [63]:
train_valid_split = df.query("dset == 'train'").shape[0] // 100 * 80
print(df.query("dset == 'train'").shape[0], train_valid_split)

75000 60000


In [64]:
train_df = df[df['dset']=='train'].iloc[:train_valid_split,:]
valid_df = df[df['dset']=='train'].iloc[train_valid_split:,:]
test_df = df[df['dset']=='test']

datasets = [train_df, train_df, test_df]

In [65]:
# print(datasets)
print(np.all([test_df['cat'].to_numpy() == 1]))
print(np.all([train_df['cat'].to_numpy() == 1]))
print(np.all([train_df['cat'].to_numpy() == 1]))

False
False
False


In [66]:
datasets = [Dataset.from_pandas(x) for x in datasets]

In [67]:
print(datasets)

[Dataset({
    features: ['dset', 'cat', 'seq', '__index_level_0__'],
    num_rows: 60000
}), Dataset({
    features: ['dset', 'cat', 'seq', '__index_level_0__'],
    num_rows: 60000
}), Dataset({
    features: ['dset', 'cat', 'seq', '__index_level_0__'],
    num_rows: 25000
})]


In [68]:
def tok_func(x): return tokenizer(" ".join(kmers(x["seq"])))

datasets = [x.map(tok_func, batched=False).rename_columns({'cat':'labels'}) for x in datasets]

  0%|          | 0/60000 [00:00<?, ?ex/s]

  0%|          | 0/60000 [00:00<?, ?ex/s]

  0%|          | 0/25000 [00:00<?, ?ex/s]

In [69]:
dds = DatasetDict({
    'train': datasets[0],
    'validation': datasets[1],
    'test':  datasets[2],
})

## 1) Fine-tuning

In [70]:

model = AutoModelForSequenceClassification.from_pretrained("simecek/DNADeberta", num_labels=2)

training_args = TrainingArguments(
    output_dir='./model',          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    overwrite_output_dir=True,      
    num_train_epochs=epochs,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=64, # the training batch size, put it as high as your GPU memory fits
    # gradient_accumulation_steps=2,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=64,  # evaluation batch size
    logging_steps=20,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=200,
    fp16=True,
    load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    save_total_limit=3,           # whether you don't have much space so you let only 5 model weights saved in the disk
# There was an error with some recursion call for push_to_hub == True
    push_to_hub=False,
    # hub_model_id="DNADeberta_fine",
    # hub_strategy="every_save"
)

def compute_metrics(eval_preds):
    # metric = load_metric("accuracy", "f1")
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dds['train'],
    eval_dataset=dds['validation'],
    tokenizer=tokenizer, 
    compute_metrics=compute_metrics,
#     early_stopping_patience - considers evaluation calls (for us, steps at the moment)
    callbacks=[EarlyStoppingCallback(early_stopping_patience = 5, early_stopping_threshold = 0.02)],
)

trainer.train()

# TODO uncomment - commented out for debug time
# model.push_to_hub(model_name + dataset_name)


loading configuration file https://huggingface.co/simecek/DNADeberta/resolve/main/config.json from cache at /home/jovyan/.cache/huggingface/transformers/23ba7a9b35a04eef3dad89d7eb23782111ad21483ca283d98e789a6ffc2a7bea.5354d2db60442ad9aec62c83c47079d6e8cc38baebd0e5e0288ab20dabc49452
Model config DebertaConfig {
  "_name_or_path": "simecek/DNADeberta",
  "architectures": [
    "DebertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": null,
  "position_biased_input": true,
  "relative_attention": false,
  "torch_dtype": "float32",
  "transformers_version": "

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Step,Training Loss,Validation Loss,Accuracy,F1
20,0.424700,0.366313,0.838517,0.794894
40,0.344200,0.354594,0.844217,0.791292
60,0.337800,0.339288,0.851517,0.801568


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The following columns in the evaluation set don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: __index_level_0__, dset, seq. If __index_level_0__, dset, seq are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 60000
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: __index_level_0__, dset, seq. If __index_level_0__, dset, seq are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 60000
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: __index_level_0__, dset, seq. If __index_level_0__, dset, seq are not expected by `

KeyboardInterrupt: 

In [ ]:
eval_metrics = trainer.evaluate()
print(eval_metrics)

In [ ]:
predictions = trainer.predict(dds['test'])
print(predictions.metrics)


In [ ]:

print(np.all(predictions.label_ids == 1))

In [ ]:
metric = load_metric("f1", "accuracy")
test_f1 = metric.compute(predictions = np.argmax(predictions.predictions, axis=-1), references = dds['test']['labels'])
print(test_f1)

metric = load_metric("accuracy", "f1")
test_acc = metric.compute(predictions = np.argmax(predictions.predictions, axis=-1), references = dds['test']['labels'])
print(test_acc)

return test_f1, test_acc